# Save a model
In the following example, we save the obtained model after each 100 epochs. The model is saved in a file with name `./my_model.ckpt`. Also, we save the last model in another file with name `./my_model_final.ckpt`.<br>
After construction, you can add additional commands to save the model.<br>

In [ ]:
import numpy as np
import tensorflow as tf

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

n_epochs = 500 #500
learning_rate = 1e-6

# Create random input and output data
inputs = np.random.randn(N, D_in)
targets = np.random.randn(N, D_out)

tf.reset_default_graph()

X = tf.constant(inputs, name="X")
y = tf.constant(targets, name="y")

# Randomly initialize weights
w1 = tf.Variable(np.random.randn(D_in, H),name="weight1")
w2 = tf.Variable(np.random.randn(H, D_out),name="weight2")

# Forward pass: compute predicted y
h = tf.matmul(X, w1, name="layer1")
h_relu = tf.maximum(h, 0)
y_pred = tf.matmul(h_relu, w2, name="layer2-predictions")

error = y_pred - y
loss = tf.reduce_sum(tf.square(error),name="squared_error")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss) # optimizer.minimize(loss,var_list=[var for var in tf.trainable_variables()])

In [ ]:
# Execution phase
session_conf=tf.ConfigProto(device_count={'GPU': 0})
sess=tf.Session(config=session_conf)

init = tf.global_variables_initializer()

saver = tf.train.Saver() # <-------------------- create a saver object

sess.run(init)
for epoch in range(n_epochs+1):
    _,loss_ = sess.run([training_op,loss]) # evaluate both in just one graph
    if epoch % 100 == 0:
        print("Epoch", epoch, "loss =", loss_)
        save_path = saver.save(sess, "./my_model.ckpt") # <-------------------- every 100 epochs
        
save_path=saver.save(sess, "./my_model_final.ckpt") # <-------------------- the last model
sess.close()

When you run `save_path = saver.save(sess, "./my_model.ckpt")`, 3 files are created:
- my_model.ckpt.data-00000-of-00001
- my_model.ckpt.index
- my_model.ckpt.meta: the saver also saves the graph structure itself in this file.

`save_path` is an optional output contains the path.<br>
In the above code, the saved model after each 100 epochs is **replaced** with a new one. You can save them seperately. see [here](https://github.com/ashkan-abbasi66/python-tricks/blob/master/README.md#tf-saveModel).<br>

# Restore a model

In [1]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

# Create some variables. ** important step **
N, D_in, H, D_out = 64, 1000, 100, 10
w1 = tf.Variable(np.random.randn(D_in, H),name="weight1")
w2 = tf.Variable(np.random.randn(H, D_out),name="weight2")

# create a saver object
saver = tf.train.Saver()

session_conf=tf.ConfigProto(device_count={'GPU': 0})
sess=tf.Session(config=session_conf)

# Restore variables from disk.
saver.restore(sess,"./my_model_final.ckpt")

print(w1)
print(w2)
# print(loss) # name 'loss' is not defined

sess.close()

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
<tf.Variable 'weight1:0' shape=(1000, 100) dtype=float64_ref>
<tf.Variable 'weight2:0' shape=(100, 10) dtype=float64_ref>


## Restore variables with a new name

Let's restore `w1` with a **different name**, such as `t1`. Note that the name of `w1` is `weight1`.

In [2]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

# Create some variables. ** important step **
N, D_in, H, D_out = 64, 1000, 100, 10
t1 = tf.Variable(np.random.randn(D_in, H),name="t1")
w2 = tf.Variable(np.random.randn(H, D_out),name="weight2")

# create a saver object
saver = tf.train.Saver(var_list={'weight1':t1})

session_conf=tf.ConfigProto(device_count={'GPU': 0})
sess=tf.Session(config=session_conf)

# Restore variables from disk.
saver.restore(sess,"./my_model_final.ckpt")

print(t1)
print(w2)
# print(w1) # name 'w1' is not defined

sess.close()

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
<tf.Variable 't1:0' shape=(1000, 100) dtype=float64_ref>
<tf.Variable 'weight2:0' shape=(100, 10) dtype=float64_ref>
